In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import datetime as dt
from dateutil.parser import parse
import numexpr as ne

In [2]:
tags = {
    "modifier": {
        "after": 20000101, 
        "before": 20050101,
        "year": None,
        "quarter": None,
        "qualifier": {"in": None, "-": None},
    },
    "key": {
        "cik": None, 
        "date": None,
        "name": None, 
        "symbol": "AAPL", 
        "unknown": ["apple", "down"]
    },
}

In [3]:
arr = [1, 2, 3, 4, 5]
print(arr.index(3))
print(arr[arr.index(5)])

2
5


In [4]:
## on home directory
# PATH = os.path.abspath(r"../data/10X/cleaned/1998/QTR2/")
# dict_dir = r"../data/10X/Loughran-McDonald_10X_Summaries_1993-2021.csv"
## on office computer
PATH = os.path.abspath(r"C:\Users\wonhyeong\workings\data\10X\cleaned")
dict_dir = r"C:\Users\wonhyeong\workings\data\10X\Loughran-McDonald_10X_Summaries_1993-2021.csv"
## on mac
# PATH = os.path.abspath(r"/Users/wonhyeong/workings/data/10X/cleaned")
# dict_dir = r"/Users/wonhyeong/workings/data/10X/Loughran-McDonald_10X_Summaries_1993-2021.csv"
summaries = pd.read_csv(dict_dir)[['CIK', 'FILING_DATE', 'ACC_NUM', 'FORM_TYPE' ,'CoName']]

In [257]:
cik = summaries.CIK.values
cik[ne.evaluate('(cik > 200000)')]
len(cik[ne.evaluate('(cik > 1000000)')])

556834

In [258]:
print(
cik[ne.evaluate('(cik > 200000)')])
print(ne.evaluate('(cik > 200000)'))

[ 861439  205239  353944 ... 1867956 1879373  711377]
[False False False ...  True  True  True]


In [259]:
a = cik[ne.evaluate(f'(cik == {1879373})')]


In [14]:
a = np.array([True, False, True, False])
b = np.array([False, True, True, False])
d = [a,b]
array = np.empty(4, dtype = bool)
array.fill(True)
for i in d:
  array *= i
print(array)

[False False  True False]


In [13]:
class file :
    def __init__(self, data_dir):
        import os
        import pandas as pd
        self.dir = data_dir
        self.dict = summaries
        self.columns = {
            "cik": self.dict.CIK.values,
            "date": self.dict.FILING_DATE.values,
            "name": self.dict.CoName.values,
            "acc": self.dict.ACC_NUM.values,
            "form": self.dict.FORM_TYPE.values,
            #"symbol": self.dict.SYMBOL.values,
        }
        self.depth = 0
        self.subdir = []
        self.file_count = []

    def index(self, dir):
        """index tree and file counts in dir
        return pandas set"""
        if not dir or (dir in ["whole", "all", "root", "tree", "node"]):
            ## print all directory tree and return listdir of root
            return os.listdir(self.dir)
        if len(dir) == 1:
            ## search inputted dir in list of subdir and return listdir of that
            if dir in self.subdir: return os.listdir(self.subdir[self.subdir.index(dir)])
        return "ERROR"
    
    def search(self, *keywords):
        """index options : 
        corpname(coName/ticker symbol/cik),
        filing date(start to end), 
        specific quarter(ex:1998-4) """
        if not keywords: return os.listdir(self.dir)
        if len(keywords) > 1: return "ERROR"
        ### search by keyword
        modifier, key = self._search_preprocess(str(keywords[0]))
        output = []
        if key: output = [x for x in os.listdir(self.dir) if x in key]
        return

        
    def _search_preprocess(self, s):
        # params : string for search 
        # return : list of keywords + tags
        keys = ['cik', 'date', 'name', 'symbol','acc', 'form']
        modifiers = ['after', 'before', 'year', 'qtr']
        a = s.split(" ")
        key = list(filter(lambda x: ':' not in x, a))
        tag = list(filter(lambda x: ':' in x, a))
        tags = dict(tuple(x.split(':')) for x in tag) if tag else {}
        tags = {k:v for k,v in tags.items() if k in key_tags}
        tags['after'] = parse(tags['after']).strftime('%Y%m%d')
        assert key or tags, "input your keyword"
        return key, tags

    def _date_parser(self, date):
        if len(date) == 4:
            return date + '0101'
        return parse(date).strftime('%Y%m%d')

    def _get_depth(self, path):
        for root, dir, file in os.walk(path):
            if file and not dir: 
                self.depth = root.count(os.sep)-path.count(os.sep)+1
        return "ERROR"
    
    def _get_subdir(self, path):
        """search for subdir in path
        return list of subdir
        every iteration, check next(os.walk(subdir))[1]
        if empty continue to next iteration"""
        return "ERROR"
    
    def _get_from_key(self, key, find, *data):
        col = self.columns[key]
        idx = ne.evaluate(f'(col == {find})')
        return idx

    def _get_from_modifier(self, modifier, value, *data):
        date = self.columns["date"]
        value = str(value)
        after = "10000000"
        before = "99999999"
        if modifier == 'after': after = value
        elif modifier == 'before': before = value
        elif modifier == 'year': 
            after = value[:4] + '0101' 
            before = value[:4] + '1231'
        elif modifier == 'qtr':
            after = value 
            before = value[:4] + str(int(value[4:6])+3) + value[6:]
        idx = ne.evaluate(f'({int(after)} < date) & (date < {int(before)})')
        return idx

In [7]:
date = file(PATH)
a = date._get_from_key("date", "19940110")
print(type(a))
print(a)
print(a[0])

<class 'numpy.ndarray'>
[False False False ... False False False]
False


In [8]:
b = date._get_from_modifier("qtr", 20200901)

In [9]:
print(b)
print(len(b))

[1171838 1387467 1622408 ... 1815086 1823524  764478]
6298
